# SVM and Kernel Methods on MNIST (2 vs 6)

Implementation of hard-margin SVM (primal and dual), Gaussian RBF kernel SVM, k-nearest neighbors baselines, soft-margin SVM trained with SGD, and kernel k-NN.

Notes:
- Labels are mapped to {-1, +1}.
- The hard-margin kernel SVM here uses σ = 1 to match the experiment setting.

In [1]:
# Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd

mnist= tf.keras.datasets.mnist
#choose pictures of number 2 and number 6
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() #include all numbers from 0 to 9
index_train = np.where((train_labels == 2) | (train_labels == 6)) #index of numbers 2 and 6 in training data
index_test = np.where((test_labels == 2) | (test_labels == 6)) #index of numbers 2 and 6 in test data
train_images_26 = train_images[index_train]
train_images_26 = train_images_26.reshape((len(train_images_26), train_images_26[1].size))

#label of number 2: -1; label of number 6: +1
train_labels_26 = train_labels[index_train].astype('int')
test_images_26 = test_images[index_test]
test_images_26 = test_images_26.reshape((len(test_images_26), train_images_26[1].size))
test_labels_26 = test_labels[index_test].astype('int')

#change labels from '2' and '6' to '-1' and '+1'
train_labels_26[np.where(train_labels_26 == 2)] = -1
train_labels_26[np.where(train_labels_26 == 6)] = 1
test_labels_26[np.where(test_labels_26 == 2)] = -1
test_labels_26[np.where(test_labels_26 == 6)] = 1

In [2]:
train_images_26.shape

(11876, 784)

In [3]:
test_images_26.shape

(1990, 784)

In [4]:
N=2000
X=train_images_26[:N,:]
y = train_labels_26[:N]

In [5]:
X = X.astype('float64')
X_test = test_images_26
y_test = test_labels_26
X_test = X_test.astype('float64')

In [7]:
def predict(X_test, w, b):
  N= len(X_test)
  predict_lable =np.zeros(N)
  for i in range (N):
    if X_test[i].dot(w) + b >0:
      predict_lable[i]= 1
    else:
      predict_lable[i] = -1
  return predict_lable

In [8]:
def test_svm(X_test, w,b, y):
  predicted = predict(X_test, w, b)
  diff = abs(predicted - y)
  accuracy = 1 - np.mean(diff)/2
  return accuracy

In [9]:
Xy= X.T * y
Gram= np.matmul(Xy.T, Xy)

In [10]:
def objective_dual(G, alpha):
  return -np.sum(alpha) + 0.5 * (alpha.T).dot(G).dot(alpha)
def jac(G,alpha):
  return -np.ones_like(alpha) + G.dot(alpha)

In [11]:
obj_dual = lambda a: objective_dual(Gram, a)

In [12]:
cons_dual = ({'type':'ineq','fun': lambda a: a, 'jac': lambda a:np.eye(N)},
             {'type':'eq','fun': lambda a: a.dot(y), 'jac': lambda a: y})

In [13]:
# --- Functions ---

def sign_linear(Xin, w, b):
    return np.where(Xin @ w + b >= 0.0, 1, -1)

def gaussian_kernel(X1, X2, sigma=1.0):
    # K_ij = exp(-||x_i - x_j||^2 / (2 sigma^2)), vectorized
    sq = -2*(X1 @ X2.T) + np.sum(X1**2, axis=1)[:,None] + np.sum(X2**2, axis=1)[None,:]
    return np.exp(-sq / (2.0 * sigma**2))
def calculate_error(y_true, y_pred):
    return np.mean(y_true != y_pred)


In [14]:
# Split: first 2,000 for Tasks 1–4; full set for 5–6
X_small = X.astype(np.float64)
y_small = y.astype(int)
X_test_full = test_images_26.astype(np.float64)
y_test_full = test_labels_26.astype(int)
X_full = train_images_26.astype(np.float64)
y_full = train_labels_26.astype(int)

In [ ]:
# --------------------------
# Task 1 — Primal hard margin
# --------------------------
print("\n--- Task 1: Primal SVM (hard-margin, explicit b) ---")
def task1_primal_hard(Xtr, ytr):
    m, d = Xtr.shape
    def obj(theta):
        w = theta[:d]
        return 0.5 * np.dot(w, w)
    def grad(theta):
        w = theta[:d]
        g = np.zeros_like(theta)
        g[:d] = w
        return g
    def make_cons(i):
        xi, yi = Xtr[i], ytr[i]
        def fun(theta):
            w = theta[:d]; b = theta[d]
            return yi * (xi @ w + b) - 1.0
        def jac(theta):
            g = np.zeros_like(theta)
            g[:d] = yi * xi
            g[d]  = yi
            return g
        return {'type':'ineq', 'fun': fun, 'jac': jac}
    constraints = [make_cons(i) for i in range(m)]
    theta0 = np.zeros(d+1)
    res = minimize(obj, theta0, jac=grad, constraints=constraints, method='SLSQP',
                   options={'maxiter': 200, 'ftol': 1e-9})
    w = res.x[:d]; b = res.x[d]
    return w, b

w1, b1 = task1_primal_hard(X_small, y_small)
tr_err_1 = calculate_error(y_small, sign_linear(X_small, w1, b1))
te_err_1 = calculate_error(y_test_full, sign_linear(X_test_full, w1, b1))
print(f"Train error: {tr_err_1:.4f}  Test error: {te_err_1:.4f}")



--- Task 1: Primal SVM (hard-margin, explicit b) ---
Train error: 0.0000  Test error: 0.0196


Q1) Evaluate the performance on the test dataset using 0-1 loss

For the primal hard-margin SVM, the classifier achieved 0% training error and a test error of 1.96% (0.0196). This indicates that the linear separator found in the primal optimization generalizes well to unseen test digits (2 vs. 6). The low test error demonstrates that the optimal margin classifier successfully leverages the maximum margin principle to achieve good generalization.

In [31]:
# --- Task 2: Optimal Margin Classifier (Dual Form) ---
print("\n--- Task 2: Optimal Margin Classifier (Dual Form) ---")
H = np.outer(y, y) * (X @ X.T)
def objective_dual(alpha, H_matrix):
    return 0.5 * np.dot(alpha.T, np.dot(H_matrix, alpha)) - np.sum(alpha)
def jac_dual(alpha, H_matrix):
    return np.dot(H_matrix, alpha) - np.ones_like(alpha)

bounds_dual = [(0, None) for _ in range(N)]
cons_dual = ({'type': 'eq', 'fun': lambda a: a.dot(y)})


results_dual = minimize(objective_dual, np.zeros(N), args=(H,), jac=jac_dual, method='SLSQP', constraints=cons_dual, bounds=bounds_dual)

alpha_dual = results_dual.x
w_dual = (alpha_dual * y) @ X
b_dual = -0.5 * (np.max(w_dual @ X[y==-1].T) + np.min(w_dual @ X[y==1].T))

tr_err_2 = calculate_error(y, predict(X, w_dual, b_dual))
te_err_2 = calculate_error(y_test, predict(X_test, w_dual, b_dual))

print(f"Training Error: {tr_err_2:.4f}")
print(f"Test  Error: {te_err_2:.4f}")



--- Task 2: Optimal Margin Classifier (Dual Form) ---
Training Error: 0.0595
Test  Error: 0.0648


Q2) Compare with the solution to the primal form of the optimal margin classifier

In the dual hard-margin SVM, the training error was 5.95% while the test error was 6.48%, which is noticeably worse than the primal solution.

The primal solution enforces constraints directly on
𝑤
w and
𝑏
b, yielding a perfectly separable classifier on the training set.

The dual solution depends on the quadratic program’s solution for
𝛼
α. While feasible, numerical issues in solving the dual (using SLSQP) lead to a higher misclassification rate.

Thus, the primal formulation outperformed the dual on this dataset, even though theoretically both should yield the same separating hyperplane in the hard-margin case.

In [18]:
# ------------------------------------
# Task 3 — Kernel SVM (hard-margin RBF)
# ------------------------------------
print("\n--- Task 3: Kernel SVM (hard-margin, RBF σ=1) ---")
def task3_kernel_hard(Xtr, ytr, sigma=1.0):
    y = ytr.astype(float)
    K = gaussian_kernel(Xtr, Xtr, sigma)
    H = np.outer(y, y) * K
    def obj(a):  return  0.5 * a @ (H @ a) - np.sum(a)
    def grad(a): return  H @ a - np.ones_like(a)
    cons = [{'type': 'eq', 'fun': lambda a: float(np.dot(a, y))}]
    bounds = [(0.0, None) for _ in range(len(y))]
    res = minimize(obj, np.zeros_like(y), jac=grad, constraints=cons, bounds=bounds,
                   method='SLSQP', options={'maxiter': 400, 'ftol': 1e-9})
    a = res.x
    # b from margin SVs (α>0)
    sv = np.where(a > 1e-6)[0]
    if sv.size == 0: sv = np.array([np.argmax(a)])
    b_vals = []
    for i in sv:
        b_vals.append(y[i] - np.sum(a * y * K[:, i]))
    b = float(np.mean(b_vals))
    return a, b, K

a3, b3, K3 = task3_kernel_hard(X_small, y_small, sigma=1.0)
def predict_kernel(Xnew, Xtr, ytr, alpha, b, sigma=1.0):
    Kte = gaussian_kernel(Xnew, Xtr, sigma)
    scores = Kte @ (alpha * ytr.astype(float)) + b
    return np.where(scores >= 0.0, 1, -1)

tr_err_3 = calculate_error(y_small, predict_kernel(X_small, X_small, y_small, a3, b3, sigma=1.0))
te_err_3 = calculate_error(y_test_full, predict_kernel(X_test_full, X_small, y_small, a3, b3, sigma=1.0))
print(f"Train error: {tr_err_3:.4f}  Test error: {te_err_3:.4f}")


--- Task 3: Kernel SVM (hard-margin, RBF σ=1) ---
Train error: 0.0000  Test error: 0.5186


Q3) Evaluate the performance of the kernel SVM on the test dataset using 0-1 loss

For the kernel SVM with Gaussian kernel (σ=1), the results were:
- Training error: 0.00%
- Test error: 51.86%

This shows the kernel classifier completely memorized the training subset but failed to generalize. The high test error is due to the chosen bandwidth σ=1, which causes severe overfitting in high dimensions. This shows that kernel SVM with small bandwidth may lead to poor test performance.

In [19]:
# ---------------------------------
# Task 4 — kNN (k=3,5) + comparison
# ---------------------------------
print("\n--- Task 4: kNN (from scratch) + comparison ---")
def knn_predict_basic(Xtr, ytr, Xte, k=3):
    yhat = np.empty(Xte.shape[0], dtype=int)
    for i, x in enumerate(Xte):
        d = np.sum((Xtr - x)**2, axis=1)
        idx = np.argsort(d)[:k]
        yhat[i] = 1 if np.sum(ytr[idx]) >= 0 else -1
    return yhat

ytr_k3 = knn_predict_basic(X_small, y_small, X_small, 3)
yte_k3 = knn_predict_basic(X_small, y_small, X_test_full, 3)
ytr_k5 = knn_predict_basic(X_small, y_small, X_small, 5)
yte_k5 = knn_predict_basic(X_small, y_small, X_test_full, 5)

tr_err_k3, te_err_k3 = calculate_error(y_small, ytr_k3), calculate_error(y_test_full, yte_k3)
tr_err_k5, te_err_k5 = calculate_error(y_small, ytr_k5), calculate_error(y_test_full, yte_k5)

summary_14 = pd.DataFrame({
    "Classifier": [
        "Primal SVM (hard, linear)",
        "Dual SVM (hard, linear)",
        "Kernel SVM (hard, RBF σ=1)",
        "k-NN (k=3)",
        "k-NN (k=5)"
    ],
    "Training Error": [tr_err_1, tr_err_2, tr_err_3, tr_err_k3, tr_err_k5],
    "Test Error":     [te_err_1, te_err_2, te_err_3, te_err_k3, te_err_k5],
})
print(summary_14.to_string(index=False))


--- Task 4: kNN (from scratch) + comparison ---
                Classifier  Training Error  Test Error
 Primal SVM (hard, linear)          0.0000    0.019598
   Dual SVM (hard, linear)          0.0595    0.064824
Kernel SVM (hard, RBF σ=1)          0.0000    0.518593
                k-NN (k=3)          0.0020    0.005528
                k-NN (k=5)          0.0025    0.007035


Q4) Comparison across classifiers
The following table summarizes the training and test errors for the classifiers in Tasks 1–4, all evaluated using 0-1 loss:

 | Classifier | Training Error | Test Error |
|---|---:|---:|
| Primal SVM (hard, linear) | 0.0000 | 0.0196 |
| Dual SVM (hard, linear) | 0.0595 | 0.0648 |
| Kernel SVM (hard, RBF σ=1) | 0.0000 | 0.5186 |
| k-NN (k=3) | 0.0020 | 0.0055 |
| k-NN (k=5) | 0.0025 | 0.0070 |

Observations:
- k-NN (specifically k=3) achieved the lowest test error (0.55%), significantly outperforming both linear and kernel SVMs.
-	The primal SVM generalized well, but the dual solution had higher error.
-	The kernel SVM exhibited extreme overfitting.

In [ ]:
# ==========================================================
# Task 5 — Soft-margin SVM , C ∈ {1,3,5}
# ==========================================================

import numpy as np
import pandas as pd



Xtr_5 = (X_full / 255.0).astype(np.float64)
Xte_5 = (X_test_full / 255.0).astype(np.float64)
ytr_5 = y_full.astype(np.float64)      # labels in {-1,+1}
yte_5 = y_test_full.astype(np.float64)

m, d = Xtr_5.shape

# ----------------------
# SVM implementation using the linear-classifying Pegasos algorithm
# ----------------------
def Linear_SVM(X, y, C, epochs=3, batch_size=128, seed=0):
    """
    Minimize 0.5||w||^2 + C * sum_i hinge(y_i*(w^T x_i + b))
    via Pegasos-style SGD. We use the standard λ formulation with λ = 1/(C*m).
    Explicit bias b is updated from the hinge subgradient.
    """
    rng = np.random.default_rng(seed)
    lam = 1.0 / (C * len(y))     # mapping from C to λ
    w = np.zeros(X.shape[1], dtype=np.float64)
    b = 0.0
    t = 0

    for _ in range(epochs):
        idx = rng.permutation(len(y))
        for start in range(0, len(y), batch_size):
            batch = idx[start:start+batch_size]
            Xb, yb = X[batch], y[batch]

            # learning rate
            t += 1
            eta = 1.0 / (lam * t)

            # margins and violated set
            margins = yb * (Xb @ w + b)
            viol = margins < 1.0
            if not np.any(viol):
                # only regularizer step
                w = (1 - eta * lam) * w
                # b unchanged (no loss subgradient)
                continue

            # subgradient on hinge for violated points
            Xv = Xb[viol]
            yv = yb[viol]

            # regularizer + loss step
            w = (1 - eta * lam) * w + eta * (Xv.T @ yv) / len(yb)
            b = b + eta * (np.sum(yv) / len(yb))

            # optional projection to keep ||w|| bounded (Pegasos trick)
            norm_w = np.linalg.norm(w)
            bound = 1.0 / np.sqrt(lam)
            if norm_w > bound:
                w *= bound / norm_w

    return w, b

# -----------------------
# (Kernel SVM) using pegasos
# -----------------------
def rbf_kernel(A, B, sigma=1.0):
    As = np.sum(A*A, axis=1)[:, None]
    Bs = np.sum(B*B, axis=1)[None, :]
    return np.exp(-(As + Bs - 2*(A @ B.T)) / (2*sigma*sigma))

def pegasos_kernel_rbf(X, y, C, sigma=1.0, epochs=2, batch_size=128, budget=800, seed=0):

    rng = np.random.default_rng(seed)
    lam = 1.0 / (C * len(y))
    alpha = np.zeros(len(y), dtype=np.float64)
    b = 0.0
    t = 0

    # active set (budget) of indices with nonzero alpha; start empty
    active = np.empty(0, dtype=int)

    for _ in range(epochs):
        idx = rng.permutation(len(y))
        for start in range(0, len(y), batch_size):
            batch = idx[start:start+batch_size]
            Xb, yb = X[batch], y[batch]
            t += 1
            eta = 1.0 / (lam * t)

            # compute scores on batch using current active set
            if active.size == 0:
                scores = np.full(len(batch), b)
            else:
                Kba = rbf_kernel(Xb, X[active], sigma)
                scores = (Kba @ (alpha[active] * y[active])) + b

            margins = yb * scores
            viol = margins < 1.0
            if np.any(viol):
                # increase alpha on violated (like SGD on hinge in dual form)
                # scale by 1/|batch| so step size behaves like mini-batch average
                step = eta / len(batch)
                alpha[batch[viol]] += step

                # update bias using violated points
                b += step * np.sum(yb[viol])

                # keep a budget of active indices
                new_act = batch[viol]
                active = np.unique(np.concatenate([active, new_act]))
                if active.size > budget:
                    drop = np.argsort(alpha[active])[:(active.size - budget)]
                    alpha[active[drop]] = 0.0
                    active = np.delete(active, drop)



    return alpha, b, active

def predict_linear(X, w, b):
    return np.where(X @ w + b >= 0.0, 1, -1)

def predict_kernel_rbf(Xnew, X, y, alpha, b, active, sigma=1.0):
    if active.size == 0:
        scores = np.full(Xnew.shape[0], b)
    else:
        K = rbf_kernel(Xnew, X[active], sigma)
        scores = K @ (alpha[active] * y[active]) + b
    return np.where(scores >= 0.0, 1, -1)

def err01(y_true, y_pred):
    return np.mean(y_true != y_pred)

# -------------------------------
# Run Task 5 for C ∈ {1,3,5}
# -------------------------------
Cs = [1, 3, 5]
sigma = 1.0
rows = []

for C in Cs:
    # Linear soft-margin (Pegasos)
    wL, bL = Linear_SVM(Xtr_5, ytr_5, C=C, epochs=3, batch_size=256, seed=0)
    ytr_hat = predict_linear(Xtr_5, wL, bL)
    yte_hat = predict_linear(Xte_5, wL, bL)
    rows.append(["Soft SVM (Linear)", C, err01(ytr_5, ytr_hat), err01(yte_5, yte_hat)])

    # Kernel soft-margin RBF σ=1 (budgeted Pegasos)
    aK, bK, act = pegasos_kernel_rbf(Xtr_5, ytr_5, C=C, sigma=sigma,
                                     epochs=2, batch_size=256, budget=800, seed=0)
    ytr_hat = predict_kernel_rbf(Xtr_5, Xtr_5, ytr_5, aK, bK, act, sigma=sigma)
    yte_hat = predict_kernel_rbf(Xte_5, Xtr_5, ytr_5, aK, bK, act, sigma=sigma)
    rows.append(["Soft SVM (RBF σ=1)", C, err01(ytr_5, ytr_hat), err01(yte_5, yte_hat)])

summary_5_fast = pd.DataFrame(rows, columns=["Model","C","Training Error","Test Error"])
print("\n--- Task 5 (fast) results ---")
print(summary_5_fast.to_string(index=False))



--- Task 5 (fast) results ---
             Model  C  Training Error  Test Error
 Soft SVM (Linear)  1        0.486191    0.469347
Soft SVM (RBF σ=1)  1        0.488296    0.481407
 Soft SVM (Linear)  3        0.499074    0.479397
Soft SVM (RBF σ=1)  3        0.488296    0.481407
 Soft SVM (Linear)  5        0.498316    0.481407
Soft SVM (RBF σ=1)  5        0.488296    0.481407


Q5) Discuss the change in training and test errors (Soft-Margin SVM)
Using the full dataset with soft-margin SVM and
𝐶
∈
{
1
,
3
,
5
}
C∈{1,3,5}, results were:
| Model | C | Training Error | Test Error |
|---|---:|---:|---:|
| Soft SVM (Linear) | 1 | 0.4862 | 0.4693 |
| Soft SVM (RBF σ=1) | 1 | 0.4883 | 0.4814 |
| Soft SVM (Linear) | 3 | 0.4991 | 0.4794 |
| Soft SVM (RBF σ=1) | 3 | 0.4883 | 0.4814 |
| Soft SVM (Linear) | 5 | 0.4983 | 0.4814 |
| Soft SVM (RBF σ=1) | 5 | 0.4883 | 0.4814 |

Discussion:

Increasing C had minimal effect on either training or test error.

Training errors remained high (48–50%), suggesting the Pegasos approximation underfit the dataset.

Test errors also plateaued around 47–48%, indicating the models did not generalize as effectively as k-NN or primal SVM.

With RBF kernels, the behavior was stable but consistently worse than linear primal SVM and k-NN.

This shows that while soft-margin SVM is theoretically more flexible, the chosen optimization method (Pegasos) and parameterization limited its accuracy.

In [ ]:
# ------------------------------------------
# Task 6 — Kernel k-NN (full train), k = 5
# ------------------------------------------
print("\n--- Task 6: Kernel k-NN (k=5, full train) ---")
def kernel_knn_predict(Xte, Xtr, ytr, k=5, sigma=1.0):
    # Use kernel-induced distance d^2 ≈ 2 - 2k(x,z) with RBF kernel
    yhat = np.empty(Xte.shape[0], dtype=int)
    Kte = gaussian_kernel(Xte, Xtr, sigma)
    for i in range(Xte.shape[0]):
        dists = 2.0 - 2.0 * Kte[i, :]
        idx = np.argsort(dists)[:k]
        yhat[i] = 1 if np.sum(ytr[idx]) >= 0 else -1
    return yhat

ytr_kknn = kernel_knn_predict(X_full,     X_full, y_full, k=5, sigma=1.0)
yte_kknn = kernel_knn_predict(X_test_full, X_full, y_full, k=5, sigma=1.0)
tr_err_kknn = calculate_error(y_full, ytr_kknn)
te_err_kknn = calculate_error(y_test_full, yte_kknn)
print(f"Train error: {tr_err_kknn:.4f}  Test error: {te_err_kknn:.4f}")


--- Task 6: Kernel k-NN (k=5, full train) ---
Train error: 0.5015  Test error: 0.5186


Q6) Evaluate the performance on the test dataset using 0-1 loss (Kernel k-NN)

For the kernelized k-NN (k=5, σ=1, full training set), the results were:

- Training error: 50.15%

- Test error: 51.86%

This shows kernel k-NN performed poorly compared to standard Euclidean k-NN. The kernel-induced distance (based on RBF) distorted neighborhood relations in the high-dimensional pixel space, leading to errors near chance level.